In [1]:
import os
import sys

import matplotlib.pyplot as plt
import torch
from omegaconf import OmegaConf, DictConfig, ListConfig

sys.path.append('../../')

from torchok.constructor.config_structure import DatasetParams
from torchok.constructor.constructor import Constructor

/opt/conda/envs/torchok/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_params = {
    'data_folder': '/workdir/rbayazitov/detection/val2017/',
    'csv_path': '/workdir/rbayazitov/detection/valid.csv'
}

transform = ListConfig([
    DictConfig({
        'name': 'Resize',
        'params': {
            'width': 224,
            'height': 224
        }
    }),
    DictConfig({
        'name': 'ToTensorV2',
        'params': {
        }
    })
    
    ]
)

daset_params = {
    'name': 'DetectionDataset',
    'params': data_params,
    'transform': transform
}

dataset_params = DatasetParams(name='DetectionDataset', params=data_params, transform=[transform])

dataset = Constructor._create_dataset(DictConfig(daset_params))
dataset

/opt/conda/envs/torchok/lib/python3.9/site-packages/albumentations/core/composition.py:53: UserWarning: transforms is single transform, but a sequence is expected! Transform will be wrapped into list.
  warnings.warn(


In [3]:
dataset[0]

[44, 67, 1, 49, 51, 51, 79, 1, 47, 47, 51, 51, 56, 50, 56, 56, 79, 57, 81]
tensor([44, 67,  1, 49, 51, 51, 79,  1, 47, 47, 51, 51, 56, 50, 56, 56, 79, 57,
        81])


{'image': tensor([[[120., 103.,  77.,  ...,  13.,  13.,  10.],
          [123.,  92.,  82.,  ...,  10.,  15.,  42.],
          [120., 100.,  73.,  ...,  14.,  14.,  23.],
          ...,
          [136., 141., 143.,  ...,  49.,  53.,  51.],
          [141., 138., 141.,  ...,  49.,  51.,  49.],
          [143., 141., 142.,  ...,  51.,  52.,  50.]],
 
         [[130., 116.,  86.,  ...,   6.,  14.,  10.],
          [135., 104.,  86.,  ...,  13.,  14.,  41.],
          [136., 102.,  78.,  ...,  16.,  12.,  26.],
          ...,
          [107., 106., 110.,  ...,  37.,  38.,  37.],
          [106., 107., 107.,  ...,  37.,  36.,  33.],
          [109., 108., 109.,  ...,  37.,  36.,  32.]],
 
         [[ 68.,  63.,  68.,  ...,  11.,  12.,  11.],
          [ 72.,  53.,  84.,  ...,  10.,  13.,  39.],
          [ 69.,  56.,  57.,  ...,  14.,  13.,  23.],
          ...,
          [ 77.,  78.,  78.,  ...,  23.,  27.,  26.],
          [ 78.,  76.,  77.,  ...,  27.,  26.,  25.],
          [ 75.,  74.,

In [ ]:
BOX_COLOR = (255, 0, 0) # Red
TEXT_COLOR = (255, 255, 255) # White


def visualize_bbox(img, bbox, class_name, color=BOX_COLOR, thickness=2):
    """Visualizes a single bounding box on the image"""
    x_min, y_min, w, h = bbox
    x_min, x_max, y_min, y_max = int(x_min), int(x_min + w), int(y_min), int(y_min + h)
   
    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=color, thickness=thickness)
    
    ((text_width, text_height), _) = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)    
    cv2.rectangle(img, (x_min, y_min - int(1.3 * text_height)), (x_min + text_width, y_min), BOX_COLOR, -1)
    cv2.putText(
        img,
        text=class_name,
        org=(x_min, y_min - int(0.3 * text_height)),
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=0.35, 
        color=TEXT_COLOR, 
        lineType=cv2.LINE_AA,
    )
    return img


def visualize(image, bboxes, category_ids, category_id_to_name):
    img = image.copy()
    for bbox, category_id in zip(bboxes, category_ids):
        class_name = str(category_id_to_name[category_id])
        img = visualize_bbox(img, bbox, class_name)
    plt.figure(figsize=(12, 12))
    plt.axis('off')
    plt.imshow(img)